In [6]:
import pandas as pd
import numpy as np
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

# GENERAL DOCUMENT MODEL

In [11]:
"""
This code sample shows Prebuilt Document operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.

To learn more, please visit the documentation - Quickstart: Form Recognizer Python client library SDKs v3.0
https://docs.microsoft.com/en-us/azure/applied-ai-services/form-recognizer/quickstarts/try-v3-python-sdk
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = "https://policyformrecognizer.cognitiveservices.azure.com/"
key = "70e2faccdcab4e1384ecc8389ff47246"

pdf_path = r"C:\Users\6130674\OneDrive - Fidelity National Financial\Desktop\invoice_1.jpg"
#formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    
with open(pdf_path, "rb") as f:
    poller = document_analysis_client.begin_analyze_document("prebuilt-document", f)
result = poller.result()

print("----Key-value pairs found in document----")
for kv_pair in result.key_value_pairs:
    if kv_pair.key and kv_pair.value:
        print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
    else:
        print("Key '{}': Value:".format(kv_pair.key.content))
            
# print("\n----Entities found in document----")
# for entity in result.entities:
#     print("Category '{}': Sub-category: '{}': Content: '{}'".format(entity.category, entity.sub_category,entity.content))

# print("----------------------------------------")

for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table.row_count, table.column_count
        )
    )
  
        
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content.encode("utf-8"),
            )
        )

print("----------------------------------------")


----Key-value pairs found in document----
Key '[Street Address]': Value:
Key '[City, ST': Value: 'ZIP]'
Key 'Phone:': Value: '(000) 000-0000'
Key 'INVOICE #': Value: '2034'
Key 'DATE': Value: '2/21/2018'
Key 'CUSTOMER ID': Value: '564'
Key 'TERMS': Value: 'Due Upon Receipt'
Key '[Name]': Value:
Key '[Company Name]': Value:
Key '[Street Address]': Value:
Key '[City, ST ZIP]': Value:
Key '[Phone]': Value:
Key '[Email Address]': Value:
Table # 0 has 8 rows and 4 columns
...Cell[0][0] has content 'b'DESCRIPTION''
...Cell[0][1] has content 'b'QTY''
...Cell[0][2] has content 'b'UNIT PRICE''
...Cell[0][3] has content 'b'AMOUNT''
...Cell[1][0] has content 'b'Service Fee''
...Cell[1][1] has content 'b'1''
...Cell[1][2] has content 'b'200.00''
...Cell[1][3] has content 'b'200.00''
...Cell[2][0] has content 'b'Labor: 5 hours at $75/hr''
...Cell[2][1] has content 'b'5''
...Cell[2][2] has content 'b'75.00''
...Cell[2][3] has content 'b'375.00''
...Cell[3][0] has content 'b'New client discount''
...

In [64]:
page_number = 0
from collections import defaultdict

keys = []
kv_pair_page_dict = defaultdict(list)
for kv_pair in result.key_value_pairs:
    page_number = kv_pair.key.bounding_regions[0].page_number
    
    if kv_pair.key and kv_pair.value:
        if kv_pair.key.bounding_regions[0].page_number == page_number and kv_pair.value.bounding_regions[0].page_number==page_number:
            kv_pair_page_dict[page_number].append((kv_pair.key.content,kv_pair.value.content))
            keys.append(kv_pair.key.content)
        print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
    else:
        if kv_pair.key.bounding_regions[0].page_number == page_number:
            kv_pair_page_dict[page_number].append((kv_pair.key.content,""))
            keys.append(kv_pair.key.content)
        print("Key '{}': Value:".format(kv_pair.key.content))

Key 'PG:': Value: '1'
Key 'COURT DIVISION:': Value: 'CIVIL'
Key 'CASE TYPE:': Value: 'FORECLOSURE'
Key 'FROM:': Value: '3/27/2022 to 3/31/2022'
Key 'JUDGE:': Value: 'ALL'
Key 'FILE DATE:': Value: '3/30/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BAKER ROSS, SUSAN'
Key 'Plaintiff:': Value: 'PENNYMAC LOAN SERVICES, LLC 3043 TOWNSGATE ROAD SUITE 200 WESTLAKE VILLAGE, CA 91361'
Key 'Plaintiff Attorney:': Value: 'BRINKMAN, MICHAEL'
Key 'Defendant:': Value: 'HEATHER BIERI 1245 GORGE BLVD AKRON, OH 44310'
Key 'FILE DATE:': Value: '3/29/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BREAUX, ALISON'
Key 'Plaintiff:': Value: 'U.S. BANK NATIONAL ASSOCIATION 4801 FREDERICA STREET OWENSBORO, KY 42301'
Key 'Plaintiff Attorney:': Value: 'COLE, AARON'
Key 'Defendant:': Value: 'HERBERT MICHAEL RAUCH AKA HERBERT M RAUCH 243 DAYTON PL AKRON, OH 44310'
Key 'FILE DATE:': Value: '3/29/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BREAUX, ALISON'
Key 

In [65]:
kv_pair_page_dict.keys()

dict_keys([1, 2])

In [63]:
from collections import defaultdict
  
kv_pair_page_dict = defaultdict(list)
for kv_pair in result.key_value_pairs:
    page_number = kv_pair.key.bounding_regions[0].page_number
    
    if kv_pair.key and kv_pair.value:
        if kv_pair.key.bounding_regions[0].page_number == page_number and kv_pair.value.bounding_regions[0].page_number==page_number:
            kv_pair_page_dict[kv_pair.key.content].append(kv_pair.value.content) 
        print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
    else:
        if kv_pair.key.bounding_regions[0].page_number == page_number:
            kv_pair_page_dict[kv_pair.key.content].append("") 
        print("Key '{}': Value:".format(kv_pair.key.content))

Key 'PG:': Value: '1'
Key 'COURT DIVISION:': Value: 'CIVIL'
Key 'CASE TYPE:': Value: 'FORECLOSURE'
Key 'FROM:': Value: '3/27/2022 to 3/31/2022'
Key 'JUDGE:': Value: 'ALL'
Key 'FILE DATE:': Value: '3/30/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BAKER ROSS, SUSAN'
Key 'Plaintiff:': Value: 'PENNYMAC LOAN SERVICES, LLC 3043 TOWNSGATE ROAD SUITE 200 WESTLAKE VILLAGE, CA 91361'
Key 'Plaintiff Attorney:': Value: 'BRINKMAN, MICHAEL'
Key 'Defendant:': Value: 'HEATHER BIERI 1245 GORGE BLVD AKRON, OH 44310'
Key 'FILE DATE:': Value: '3/29/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BREAUX, ALISON'
Key 'Plaintiff:': Value: 'U.S. BANK NATIONAL ASSOCIATION 4801 FREDERICA STREET OWENSBORO, KY 42301'
Key 'Plaintiff Attorney:': Value: 'COLE, AARON'
Key 'Defendant:': Value: 'HERBERT MICHAEL RAUCH AKA HERBERT M RAUCH 243 DAYTON PL AKRON, OH 44310'
Key 'FILE DATE:': Value: '3/29/2022'
Key 'CASE TYPE:': Value:
Key 'FORECLOSURE JUDGE:': Value: 'BREAUX, ALISON'
Key 

In [ ]:
pd.concat({k: pd.DataFrame(v) for k, v in kv_pair_page_dict.items()}, axis=1)

In [66]:
kv_pair_page_dict

defaultdict(list,
            {1: [('PG:', '1'),
              ('COURT DIVISION:', 'CIVIL'),
              ('CASE TYPE:', 'FORECLOSURE'),
              ('FROM:', '3/27/2022 to 3/31/2022'),
              ('JUDGE:', 'ALL'),
              ('FILE DATE:', '3/30/2022'),
              ('CASE TYPE:', ''),
              ('FORECLOSURE JUDGE:', 'BAKER ROSS, SUSAN'),
              ('Plaintiff:',
               'PENNYMAC LOAN SERVICES, LLC 3043 TOWNSGATE ROAD SUITE 200 WESTLAKE VILLAGE, CA 91361'),
              ('Plaintiff Attorney:', 'BRINKMAN, MICHAEL'),
              ('Defendant:', 'HEATHER BIERI 1245 GORGE BLVD AKRON, OH 44310'),
              ('FILE DATE:', '3/29/2022'),
              ('CASE TYPE:', ''),
              ('FORECLOSURE JUDGE:', 'BREAUX, ALISON'),
              ('Plaintiff:',
               'U.S. BANK NATIONAL ASSOCIATION 4801 FREDERICA STREET OWENSBORO, KY 42301'),
              ('Plaintiff Attorney:', 'COLE, AARON'),
              ('Defendant:',
               'HERBERT MICHAEL R

In [54]:
list_keys = np.unique(keys)

In [55]:
list_keys

array(['CASE', 'CASE FORECLOSURE', 'CASE TYPE:',
       'CASE TYPE: FORECLOSURE JUDGE:', 'COURT DIVISION:', 'DROP:',
       'Defendant:', 'FILE DATE:', 'FORECLOSURE JUDGE:', 'FROM:',
       'JUDGE:', 'PG:', 'Plaintiff Attorney:', 'Plaintiff:'], dtype='<U29')

In [56]:
list_keys = np.delete(list_keys,[0,1,3,5,8,9,11])

In [73]:
kv_pair_page_dict.keys()

dict_keys([1, 2])

In [91]:
from collections import defaultdict

updated_dict =defaultdict(list)
discarded_dict =defaultdict(list)
for v in kv_pair_page_dict.values():
    for i in v:
        if i[0] in list_keys:
            if i[0] == 'CASE TYPE: FORECLOSURE JUDGE:':
                i[0].split()
                updated_dict[i[0]].append(i[1])
        else:
            discarded_dict[i[0]].append(i[1])

In [83]:
dataframe_dict = {k: v for k, v in updated_dict.items()}

In [ ]:
for page, key_value_pair in kv_pair_page_dict.items():
    df = pd.DataFrame(data=key_value_pair, columns=["Key", "Value"])
    df.to_excel(f"Key_Value_Page_{page}.xlsx",index=False)
        
    

# LAYOUT MODEL 

In [3]:
endpoint = "https://policyformrecognizer.cognitiveservices.azure.com/"
key = "70e2faccdcab4e1384ecc8389ff47246"

#formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
#pdf_path = r"C:\Users\6130674\OneDrive - Fidelity National Financial\Desktop\summit co courts 03.30.2022-4.4.2022.pdf"
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

pdf_path = r"C:\Users\6130674\OneDrive - Fidelity National Financial\Desktop\invoice_1.jpg"

with open(pdf_path, "rb") as f:
    poller = document_analysis_client.begin_analyze_document("prebuilt-layout", f)
result = poller.result()

for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
         "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

for page in result.pages:
    for line_idx, line in enumerate(page.lines):
        print(
         "...Line # {} has text content '{}'".format(
        line_idx,
        line.content.encode("utf-8")
        )
    )

    for selection_mark in page.selection_marks:
        print(
         "...Selection mark is '{}' and has a confidence of {}".format(
         selection_mark.state,
         selection_mark.confidence
         )
    )

for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table.row_count, table.column_count
        )
    )
  
        
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content.encode("utf-8"),
            )
        )

print("----------------------------------------")

...Line # 0 has text content 'b'[Company Name]''
...Line # 1 has text content 'b'[Street Address]''
...Line # 2 has text content 'b'[City, ST ZIP]''
...Line # 3 has text content 'b'Phone: (000) 000-0000''
...Line # 4 has text content 'b'BILL TO''
...Line # 5 has text content 'b'[Name]''
...Line # 6 has text content 'b'[Company Name]''
...Line # 7 has text content 'b'[Street Address]''
...Line # 8 has text content 'b'[City, ST ZIP]''
...Line # 9 has text content 'b'[Phone]''
...Line # 10 has text content 'b'[Email Address]''
...Line # 11 has text content 'b'INVOICE''
...Line # 12 has text content 'b'INVOICE #''
...Line # 13 has text content 'b'DATE''
...Line # 14 has text content 'b'2034''
...Line # 15 has text content 'b'2/21/2018''
...Line # 16 has text content 'b'CUSTOMER ID''
...Line # 17 has text content 'b'TERMS''
...Line # 18 has text content 'b'564''
...Line # 19 has text content 'b'Due Upon Receipt''
...Line # 20 has text content 'b'DESCRIPTION''
...Line # 21 has text content '

In [5]:
pdf_path

'C:\\Users\\6130674\\OneDrive - Fidelity National Financial\\Desktop\\invoice_1.jpg'

## Converting into Table and Saving in Excel 

In [12]:
pdf_file_name = os.path.basename(pdf_path)
# os.makedirs(pdf_file_name)
df_dict = {}
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table.row_count, table.column_count
        )
    )
    df = pd.DataFrame(index=np.arange(table.row_count), columns=np.arange(table.column_count))
        
    for cell in table.cells:
        df.iloc[cell.row_index,cell.column_index] = cell.content
        print(
            "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content,
            )
        )
    df_dict["Table_No."+str(table_idx+1)] = df    
    
#     df.to_csv(f'{pdf_file_name}\Table_Page_No.{table_idx+1}.txt', index=False,header=None, sep = " ")    

print("----------------------------------------")

Table # 0 has 8 rows and 4 columns
...Cell[0][0] has content 'DESCRIPTION'
...Cell[0][1] has content 'QTY'
...Cell[0][2] has content 'UNIT PRICE'
...Cell[0][3] has content 'AMOUNT'
...Cell[1][0] has content 'Service Fee'
...Cell[1][1] has content '1'
...Cell[1][2] has content '200.00'
...Cell[1][3] has content '200.00'
...Cell[2][0] has content 'Labor: 5 hours at $75/hr'
...Cell[2][1] has content '5'
...Cell[2][2] has content '75.00'
...Cell[2][3] has content '375.00'
...Cell[3][0] has content 'New client discount'
...Cell[3][1] has content ''
...Cell[3][2] has content '(50.00)'
...Cell[3][3] has content '(50.00)'
...Cell[4][0] has content ''
...Cell[4][1] has content ''
...Cell[4][2] has content ''
...Cell[4][3] has content ''
...Cell[5][0] has content ''
...Cell[5][1] has content ''
...Cell[5][2] has content ''
...Cell[5][3] has content ''
...Cell[6][0] has content ''
...Cell[6][1] has content ''
...Cell[6][2] has content ''
...Cell[6][3] has content ''
...Cell[7][0] has content ''
.

In [13]:
from pandas import ExcelWriter
def save_xls(dict_df, path):
    writer = ExcelWriter(path)
    for key in dict_df:
        dict_df[key].to_excel(writer, key, index=False, header=False)
    writer.save()    

In [14]:
save_xls(df_dict,f"./{pdf_file_name}.xlsx")

# CUSTOM MODEL


In [ ]:
# import module
from pdf2image import convert_from_path, convert_from_bytes
pdf_path = r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\Annotation\WF\1.WF-7778 03-28 NORMAL.pdf"
images = convert_from_bytes(open(pdf_path, "rb").read())
os.makedirs(pdf_path+"_images",exist_ok=True)
for i, image in enumerate(images):
    
    image.save(f'{pdf_path}_images/image_{i}.jpg', 'JPEG')
    #pages[i].save('page'+ str(i) +'.jpg', 'JPEG')
#formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"


In [ ]:
import glob
list_images = [file for file in glob.glob(f'{pdf_path}_images/*.jpg')]

In [2]:
"""
This code sample shows Custom Model operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.
`
To learn more, please visit the documentation - Quickstart: Form Recognizer Python client library SDKs v3.0
https://docs.microsoft.com/en-us/azure/applied-ai-services/form-recognizer/quickstarts/try-v3-python-sdk
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient,DocumentField
from collections import defaultdict
endpoint = "https://eastus2.api.cognitive.microsoft.com/"
key = "132d0d0fb8474520a464887a31f356dc"
# model_id = "Compose_model_12_may_2022"
model_id = "Mrcooper_Chase_Neural_Compose"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
# DocumentField()

pdf_path = r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\2 CHASE\2 CHASE\23. Chase 1105 2_2.pdf"

with open(pdf_path, "rb") as fd:
    document = fd.read()
# for image in list_images:
#     with open(image, "rb") as fd:
#         im = fd.read()
poller = document_analysis_client.begin_analyze_document(model=model_id, document=document)
result = poller.result()



results_dict= defaultdict(list)
for idx, document in enumerate(result.documents):
        for name, field in document.fields.items():

            field_value = field.value if field.value else field.content
            #print("......found Key Name '{}' with value '{}' and with confidence {}".format(name, field_value, field.confidence))
            
            results_dict[name].append(field_value)
#             results_dict[name].append(field.confidence)

print(results_dict)            
    # # Make sure your document's type is included in the list of document types the custom model can analyze
    # poller = document_analysis_client.begin_analyze_document_from_url(model_id, formUrl)
    # result = poller.result()

    # for idx, document in enumerate(result.documents):
    #     print("--------Analyzing document #{}--------".format(idx + 1))
    #     print("Document has type {}".format(document.doc_type))
    #     print("Document has confidence {}".format(document.confidence))
    #     print("Document was analyzed by model with ID {}".format(result.model_id))
    #     for name, field in document.fields.items():
    #         field_value = field.value if field.value else field.content
    #         print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))


#     # iterate over tables, lines, and selection marks on each page
#     for page in result.pages:
#         print("\nLines found on page {}".format(page.page_number))
#         for line in page.lines:
#             print("...Line '{}'".format(line.content.encode('utf-8')))
#         for word in page.words:
#             print(
#                 "...Word '{}' has a confidence of {}".format(
#                     word.content.encode('utf-8'), word.confidence
#                 )
#             )
#         for selection_mark in page.selection_marks:
#             print(
#                 "...Selection mark is '{}' and has a confidence of {}".format(
#                     selection_mark.state, selection_mark.confidence
#                 )
#             )

# # for i, table in enumerate(result.tables):
#     print("\nTable {} can be found on page:".format(i + 1))
#     for region in table.bounding_regions:
#         print("...{}".format(i + 1, region.page_number))
#     for cell in table.cells:
#         print(
#             "...Cell[{}][{}] has content '{}'".format(
#                 cell.row_index, cell.column_index, cell.content.encode('utf-8')
#             )
#         )
# print("-----------------------------------")


defaultdict(<class 'list'>, {'LenderName': ['CHASE O'], 'PayoffStatement': ['This payoff quote is good through 03/04/2022 at 6 p.m. Eastern Time. For more information about paying off your mortgage, please visit chase.com/payoff. Principal Balance: Interest from 02/01/2022 to 03/04/2022: Recording Cost :* $297,243.82 $735.73 $100.00'], 'PageNumber': ['Page 3 of 7'], 'LoanNumber': ['1434081105'], 'WiringStatement': ["Wire Transfer (fastest option) To process it the same day, we need the full payoff amount by 6 p.m. Eastern Time. Include the following information with your wire transfer: JPMorgan Chase Bank, N.A. Account number: 323553729 Routing number: 021000021 Account name: Chase Payoff Wire Account OBI text: 1434081105 Attn: Payoff Processing Your name Your property address Your closing agent's contact information"], 'TotalDueAmount': ['Total Amount Due: $298,079.55'], 'Mortagagor': ['Robert P. Zvara'], 'CoMortagagor': ['Zvara#Nicole#R'], 'PropertyAddress': ['255 Hazelton Court Morg

In [ ]:
import os

In [ ]:
def efax_pdf_to_text(directory_of_pdfs):
    for pdf_file in glob.glob(os.path.join(directory_of_pdfs,"*.pdf")):
        pdf_file_name = os.path.basename(pdf_file)  
        print(pdf_file_name)

In [ ]:
efax_pdf_to_text(r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\Annotation\WF")

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
endpoint = "https://eastus2.api.cognitive.microsoft.com/"
key = "946e5a8d6ca3400eb0b1c0398b92105c"
model_id = "Neural_Net_Model_Batch_2_19_May"

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient,DocumentField
from collections import defaultdict
import glob
import os
endpoint = "https://eastus2.api.cognitive.microsoft.com/"
key = "9e12a42689cb4afd996ed992fb1f07d7"
model_id = "Neural_Net_Model_Batch_3_19_May"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
# DocumentField()

def efax_pdf_to_text(directory_of_pdfs):
    final_information = {}

    for pdf_file in tqdm_notebook(glob.glob(os.path.join(directory_of_pdfs,"*.pdf"))):
        pdf_file_name = os.path.basename(pdf_file)
        print(pdf_file_name)

        with open(pdf_file, "rb") as fd:
            document = fd.read()
        poller = document_analysis_client.begin_analyze_document(model=model_id, document=document)
        result = poller.result()
        results_dict= defaultdict(list)
        no_text_fields = ['TotalOutstandingAmount','WiringStatements','PayoffStatement']
#         for idx, document in enumerate(result.documents):
#                 for name, field in document.fields.items():

#                     field_value = field.value if field.value else field.content

#                     if name in no_text_fields:
#                         if len(field_value)>0:
#                             results_dict[name].append("True")
#                         else:
#                             results_dict[name].append("False")
#                     else:
#                         results_dict[name].append(field_value)
        for idx, document in enumerate(result.documents):
            for name, field in document.fields.items():

                field_value = field.value if field.value else field.content
                #print("......found Key Name '{}' with value '{}' and with confidence {}".format(name, field_value, field.confidence))

                results_dict[name].append(field_value)
                results_dict[name+"_Confidence"].append(field.confidence)
        final_information[pdf_file_name] = results_dict
    
    return final_information

In [ ]:
data_1= efax_pdf_to_text(r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\batch_1")

In [ ]:
data_2 = efax_pdf_to_text(r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\batch_2_P2")

In [ ]:
data_2_1 = efax_pdf_to_text(r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\batch_2_P1")

In [ ]:
data_3_2 = efax_pdf_to_text(r"C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\batch_3_P2")

In [ ]:
def convert_data_to_dataframe(json_data):
    data_frame = pd.concat({k: pd.DataFrame(v) for k, v in json_data.items()}, axis=0)
    data_frame = data_frame.reset_index().drop(["level_1"], axis=1).rename({"level_0":"Filename"}, axis=1)
    first_column = data_frame.pop('Filename')
    data_frame.insert(0, 'Filename', first_column)
    return data_frame

In [ ]:
def save_to_json(data):
    import json

    with open(r'C:\Users\6130674\OneDrive - Fidelity National Financial\Documents\New_Project\batch_3_P2\data.json', 'w') as fp:
        json.dump(data, fp)

In [ ]:
save_to_json(data_3_2)

In [ ]:
import pandas as pd

In [ ]:
dataframe = convert_data_to_dataframe(data_1)

In [ ]:
dataframe_2_1 = convert_data_to_dataframe(data_2)

In [ ]:
dataframe_2_2 = convert_data_to_dataframe(data_2_1)

In [ ]:
dataframe_3_1 = convert_data_to_dataframe(data_3_1)

In [ ]:
dataframe_3_2 = convert_data_to_dataframe(data_3_2)

In [ ]:
combined_dataframe = pd.concat([dataframe,dataframe_2_1,dataframe_2_2, dataframe_3_1,dataframe_3_2])

In [ ]:
combined_dataframe["PageNumber"] = combined_dataframe["PageNumber"].str.findall(r'\d+').str[-1]

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
combined_dataframe.rename({"PageNumber":"TotalPages", "PageNumber_Confidence":"TotalPages_Confidence"},inplace=True)

In [ ]:
combined_dataframe.sort_values("Filename",inplace=True)

In [ ]:
combined_dataframe.shape

In [ ]:
old_data = pd.read_excel("eFax_Neural_Net_Compose_model_output_final_output.xlsx")

In [ ]:
data_11=pd.read_excel("eFax_Neural_Net_Compose_model_output_Batch_1.xlsx")
data_12 = pd.read_excel("eFax_Neural_Net_Compose_model_output_Batch_2.xlsx")
data_13= pd.read_excel("eFax_Neural_Net_Compose_model_output_Batch_3.xlsx")

In [ ]:
first_iteration_combined = pd.concat([data_11,data_12,data_13])

In [ ]:
final_data = pd.concat([first_iteration_combined,combined_dataframe])

In [ ]:
final_data.sort_values("Filename",inplace=True)

In [ ]:
final_data.to_excel("eFax_Neural_Net_Compose_model_output_final_output_.xlsx", index=False)